In [8]:
import torch
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
%matplotlib inline
from torch.utils.data import Dataset, DataLoader
from data_provider.data_factory import data_provider

from tqdm.notebook import tqdm

from sklearn.metrics import r2_score, mean_squared_error
import datetime

from autoformer_inference import Autoformer,autoformer_predict

In [ ]:
setting='qber_96_48_60_loginf_t_non_outliers_Autoformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el6_dl6_df2048_fc1_ebtimeF_dtFalse_Exp_0'
# setting='qber_96_48_60_loginf_t_non_outliers_pred_3_Autoformer_custom_ftMS_sl96_ll48_pl3_dm512_nh8_el6_dl8_df2048_fc1_ebtimeF_dtFalse_Exp_0'
# setting='qber_96_48_60_loginf_t_outliers_Autoformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el6_dl6_df2048_fc1_ebtimeF_dtFalse_Exp_0'

path=f'./checkpoints/{setting}/checkpoint.pth'

seq_len=96
label_len=48
# pred_len=1
pred_len=1

model = Autoformer(
    dropout=0.05,
    freq='t',
    seq_len=seq_len,
    label_len=label_len,
    pred_len=pred_len,
    e_layers=6,
    d_layers=6,
    # d_layers=8,
    n_heads=8,
    factor=1,
    enc_in=6,
    dec_in=6,
    c_out=6,
    d_model=512,
    d_ff=2048,
    moving_avg=25,
    output_attention=True,
    embed='timeF',
    activation='gelu',)

model.load_state_dict(torch.load(path))

device='cuda'
model.to(device)

In [ ]:
df = pd.read_csv('datasets/non_outliers_step=60_log_inf_test=0.1.csv')
# df = pd.read_csv('datasets/outliers_step=60_log_inf_test=0.1.csv')

In [ ]:
def get_ds(index,start_ts=0,step=60):
    index_tmp=index*step+start_ts
    date=datetime.datetime.fromtimestamp(index_tmp).strftime('%Y-%m-%d %H:%M:%S')
    date=pd.Timestamp(date)
    return date

def norm(value):
    if value<0.005:
        return 0.0108-0.0023
    else:
        return value

df=pd.read_csv('datasets/data.csv')

df['index']=range(df.__len__())
df['date']=df['index'].map(get_ds)

del df['id']

df['e_mu_current']=df['e_mu_current'].map(norm)
df['e_mu_current']=np.log(df['e_mu_current'])



In [7]:
df

,e_mu_current,e_nu_1,e_nu_2,q_mu,q_nu1,q_nu2,date
0,-4.645992,0.01545,0.19912,0.563805,0.171919,0.006965,1970-04-26 21:10:00
1,-4.746734,0.01331,0.16451,0.563071,0.168300,0.007908,1970-04-26 21:11:00
2,-4.590282,0.01432,0.18310,0.562257,0.169504,0.008020,1970-04-26 21:12:00
3,-4.650168,0.01214,0.12969,0.567922,0.167161,0.007433,1970-04-26 21:13:00
4,-4.718339,0.01117,0.12858,0.564888,0.170380,0.007933,1970-04-26 21:14:00
...,...,...,...,...,...,...,...
18155,-4.777145,0.02085,0.25589,0.566484,0.172819,0.008164,1970-05-09 11:45:00
18156,-4.634599,0.01592,0.23668,0.572031,0.170474,0.007907,1970-05-09 11:46:00
18157,-4.726209,0.01590,0.24216,0.569204,0.173147,0.007366,1970-05-09 11:47:00
18158,-4.746734,0.01384,0.24447,0.570114,0.168854,0.007378,1970-05-09 11:48:00


In [ ]:
model.eval()

preds = []
trues = []

with torch.no_grad():
    for i  in tqdm(range(len(df)-seq_len)):

        df_tmp=df.iloc[i:i+seq_len]

        pred = autoformer_predict(model, df_tmp,pred_len=pred_len)[0]

        preds.append(pred)

